In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [2]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

def replace_strings(data,name):
    for i in range(len(data)):
        if(data[name].iloc[i].find(',')!=-1):
            data[name].iloc[i]=data[name].iloc[i].split(",")[-1]
    return data

In [3]:
data=pd.read_excel("U:/JIRA/JIRAExport/train_data.xls")
data=data.sample(frac=1.0, replace=True)
data.head()

,Issue Key,Summary,Status,Created,Component/s,Description,Resolved Date
1783,NCG3D-24995,Quick Search is not working for iAP1 iPod devi...,Closed,2016-09-19 14:21:29,SW_INFOTAINMENT,h4. Test Environment :\n* HW Sample : C Sample...,2016-10-14 11:36:40
44177,NCG3D-90394,AIVIBP-10168 [System]It shows blank screen [By...,Closed,2018-03-09 14:09:32,SW_SWUPDATE,{color:red} !!! Customer Ticket !!! {color} \n...,2018-07-07 13:40:49
74520,NCG3D-20777,city center icon is missing in Destination screen,Closed,2016-07-25 08:30:35,SW_NAVIGATION,1. sk_destination -> sk_next page\n\nObservati...,2016-11-02 06:06:03
62319,NCG3D-203,USB (FFW): After reaching end of the song (via...,Closed,2015-05-29 12:26:43,"SW_FRAMEWORK_HMI,SW_INFOTAINMENT",[Initial State] Active source is USB (media)\n...,2015-07-07 07:50:36
77824,NCG3D-89994,AIVIBP-10008 [Phone/SMS] [Favorite(Connect Fir...,Closed,2018-03-08 12:06:21,SW_PHONE,{color:red} !!! Customer Ticket !!! {color} \n...,2018-09-07 03:55:09


In [4]:
data=data.dropna()
data=replace_strings(data,'Component/s')


C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
Comp=list(data['Component/s'].unique())
train=data.sample(frac=0.90,random_state=120)
test=data.drop(train.index)
data=train

In [6]:
stemmer = SnowballStemmer("german")
original_words = ['careless', 'errors', 'does', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted','Gut','auffallender']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,careless,careless
1,errors,error
2,does,does
3,mules,mul
4,denied,denied
5,died,died
6,agreed,agreed
7,owned,owned
8,humbled,humbled
9,sized,sized


In [7]:
document_num = 50
doc_sample = data.iloc[10]['Summary']

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['LVDS', 'Return-Loss', 'DTV']


Tokenized and lemmatized document: 
['lvds', 'return', 'loss']


In [8]:
processed_docs = []
i=0
for doc in data['Summary']:
    processed_docs.append(preprocess(doc))
   

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [10]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 action
1 button
2 categori
3 click
4 continus
5 menu
6 pag
7 view
8 follow
9 implement
10 instruction


In [11]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [13]:
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 90 ("slideshow") appears 1 time.
Word 91 ("station") appears 1 time.
Word 92 ("zon") appears 1 time.


In [14]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics =len(Comp), 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.198*"camera" + 0.189*"appear" + 0.087*"remov" + 0.051*"restart" + 0.045*"paramet" + 0.043*"battery" + 0.041*"voltag" + 0.040*"clear" + 0.026*"performanc" + 0.025*"scor"


Topic: 1 
Words: 0.436*"list" + 0.158*"availabl" + 0.117*"option" + 0.080*"station" + 0.041*"highlight" + 0.036*"select" + 0.025*"spac" + 0.021*"show" + 0.015*"charg" + 0.009*"instead"


Topic: 2 
Words: 0.505*"popup" + 0.096*"properly" + 0.058*"action" + 0.053*"disclaim" + 0.042*"exist" + 0.039*"show" + 0.037*"align" + 0.032*"hk_back" + 0.020*"half" + 0.018*"vics"


Topic: 3 
Words: 0.442*"press" + 0.192*"short" + 0.086*"upp" + 0.068*"low" + 0.052*"degrad" + 0.050*"navigation" + 0.028*"cas" + 0.010*"hid" + 0.008*"transf" + 0.007*"strg"


Topic: 4 
Words: 0.170*"data" + 0.154*"servic" + 0.124*"respons" + 0.083*"fuel" + 0.048*"fold" + 0.048*"brows" + 0.034*"memory" + 0.021*"select" + 0.020*"eject" + 0.019*"disc"


Topic: 5 
Words: 0.289*"auto" + 0.207*"android" + 0.194*"vehicl" + 0.054*"steer" + 0.04

In [26]:
# Data preprocessing step for the unseen document
result=[]
for each in range(len(test)):
    num = each
    print(num)
    unseen_document = test.iloc[num]['Summary']
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    i=0
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        test_class=i
        i=i+1
    result.append(test.iloc[num]['Component/s']==Comp[test_class])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210


In [30]:
(result.count(True)/len(result))*100

7.767565965016305

In [33]:
unseen_document = 'Reebot problem'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
i=0
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    test_class=i
    i=i+1
print(test.iloc[num]['Component/s']==Comp[test_class])

True


In [31]:
len(result)

3373

In [27]:
len(test)

3373

In [28]:
len(result)

3373